# Achraf JDAY 4/12/2022

### Installing libraries

In [12]:
!python3 -m pip install click
!python3 -m pip install requests
!python3 -m pip install csv
!python3 -m pip install bs4
!python3 -m pip install datetime

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.
Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.
Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.
Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.
Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


### Importing libraries

In [7]:
import requests
from bs4 import BeautifulSoup
import csv
import datetime
import click

### Source Code

In [8]:
class crawler:
    def __init__(self, url):
        self.url = url
        self.links = []
        self.links_with_error_404 = []
        self.links_with_authentication_401 = []
        self.links_scanned = []

    def isValidUrl(self,sous_url:str)->bool:

        if sous_url not in self.links and sous_url != "#" and sous_url.startswith(self.url):
            return True
        else :
            return False


    def getPage(self) -> requests.models.Response:

        page = requests.get(self.url)
        return page


    def retrieveLinks(self):

        bs = BeautifulSoup(self.getPage().content, "html.parser")
        web_links = bs.select('a')
        actual_web_links = [web_link['href'] for web_link in web_links]
        for elt in actual_web_links :
            if self.isValidUrl(elt) :
                self.links.append(elt)
        return

    def exportInCsvFile(self, liste_a_exporter:list):

        print("Création du fichier CSV.")
        with open("links.csv", 'w',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Liste horodatée GMT du %s" % datetime.datetime.utcnow()]) 
            for i in range(0, len(liste_a_exporter)):
                writer.writerow([liste_a_exporter[i]]) 

        return

    def filterLinks404(self):

        for i in range(0,len(self.links)):
            try:
                response = requests.get(self.links[i])
                if response.status_code == 404:
                    self.links_with_error_404.append(self.links[i])
            except :
                print("Exception Error")
        print("Vous avez %d lien(s) 404." %len(self.links_with_error_404))

        return

    def filterListAuthenticate401(self):

        for i in range(0, len(self.links)):
            try:
                authi = requests.get(self.links[i], auth=("",""))
                if authi.status_code == 401:
                    self.links_with_authentication_401.append(self.links[i])
            except:
                print("Exception Error : lien %s" %(self.links[i]))
        print("Vous avez %d lien(s) avec authentification." %len(self.links_with_authentication_401))
        return

    def retrieveAbsoluteAllLinks(self):
        self.retrieveLinks() 
        self.links_scanned.append(self.url) 
        indice_initial = 0
        indice_final = len(self.links)-1
        print("L'indice initial est %d, le final est %d" %(indice_initial,indice_final))


        while len(self.links) != len(self.links_scanned): 
            print("Nbr de liens : %d. Nbr de liens scannés : %d." %(len(self.links), len(self.links_scanned)))
            for i in range(indice_initial,indice_final):
                try:
                    print("Scan du lien %d. Liens totaux = %d" %(i,len(self.links)))
                    page = requests.get(self.links[i]) 
                    bs = BeautifulSoup(page.content, "html.parser")
                    web_links = bs.select('a')
                    actual_web_links = [web_link['href'] for web_link in web_links]
                    for elt in actual_web_links:
                        if self.isValidUrl(elt):
                            self.links.append(elt)  
                    self.links_scanned.append(self.links[i]) 
                    indice = i+1
                except:
                    print("Le lien %s ne peut être lu." %self.links[i])
                    self.links_scanned.append(self.links[i])
                    indice = i+1
            indice_initial = indice
            indice_final = len(self.links)-1
        print("Scan de %s terminé. %d liens trouvés au total. %d liens scannés." %(self.url, len(self.links), len(self.links_scanned)))
        return

In [10]:
@click.command()
@click.option('--error404', default=False, help='Error404 links.')
@click.option('--url', help= "Enter the crawlable URL")
@click.option('--auth', default=False, help = "Authentication links.")
@click.option('--export',default=False, help = "Export in a csv file the list of links.")
def crawl(url, error404,export,auth):

    site_a_crowler = crawler(url)
    site_a_crowler.retrieveAbsoluteAllLinks()
    if error404 :
        site_a_crowler.filterLinks404()
        if export :
            site_a_crowler.exportInCsvFile(site_a_crowler.links_with_error_404)
        else:
            click.echo(site_a_crowler.links_with_error_404)
            print(site_a_crowler.links_with_error_404)
    elif auth :
        site_a_crowler.filterListAuthenticate401()
        if export:
            site_a_crowler.exportInCsvFile(site_a_crowler.links_with_authentication_401)
        else:
            print(site_a_crowler.links_with_authentication_401)
    else:
        if export:
            site_a_crowler.exportInCsvFile(site_a_crowler.links_scanned)
        else:
            print(site_a_crowler.links_scanned)
    return



### Testing

In [ ]:
crawl(url = 'www.aforp.fr' , error404 = True , export = True)

In [ ]:
crawl(url='www.aforp.fr' , auth = True , export = True)

You can also export the code to a .py file to test it on the command line as follows:

    python3 Click2.py --url='www.aforp.fr' --error404=True --export=True
    python3 Click2.py --url='www.aforp.fr' --auth=True --export=True